In [63]:
import numpy as np
import pandas as pd


from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from lightgbm import LGBMRegressor

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submit = pd.read_csv('submit.csv')

In [3]:
train.head()

,student_id,gender,race/ethnicity,parental level of education,lunch,test preparation course,score,parental_job_status,parental_wealth_classification
0,1014,female,group D,high school,standard,completed,88,employed,poor
1,1650,female,NaN,some high school,free/reduced,completed,34,unemployed,middle class
2,717,female,group C,some high school,standard,none,59,employed,middle class
3,258,female,group D,NaN,standard,none,68,employed,poor
4,1312,male,group D,some college,standard,completed,74,employed,poor


In [4]:
test.head()

,student_id,gender,race/ethnicity,parental level of education,lunch,test preparation course,parental_job_status,parental_wealth_classification
0,810,female,group A,some college,standard,none,employed,middle class
1,1609,female,NaN,associate's degree,free/reduced,NaN,employed,poor
2,15,female,NaN,some high school,NaN,none,NaN,middle class
3,134,male,group D,NaN,standard,none,employed,wealthy
4,846,male,group C,associate's degree,standard,completed,employed,poor


In [5]:
train['gender'].fillna('female', inplace=True)

In [6]:
test['gender'].fillna('female', inplace=True)

In [7]:
train['race/ethnicity'].fillna('group A', inplace=True)

In [8]:
test['race/ethnicity'].fillna('group A', inplace=True)

In [9]:
train['parental level of education'].fillna("master's degree", inplace=True)

In [10]:
test['parental level of education'].fillna("master's degree", inplace=True)

In [11]:
train['lunch'].fillna("free/reduced", inplace=True)

In [12]:
test['lunch'].fillna("free/reduced", inplace=True)

In [13]:
train['test preparation course'].fillna('completed', inplace=True)

In [14]:
test['test preparation course'].fillna('completed', inplace=True)

In [15]:
train['parental_job_status'].fillna('unemployed', inplace=True)

In [16]:
test['parental_job_status'].fillna('unemployed', inplace=True)

In [17]:
train['parental_wealth_classification'].fillna('wealthy', inplace=True)

In [18]:
 test['parental_wealth_classification'].fillna('wealthy', inplace=True)

In [19]:
train.head()

,student_id,gender,race/ethnicity,parental level of education,lunch,test preparation course,score,parental_job_status,parental_wealth_classification
0,1014,female,group D,high school,standard,completed,88,employed,poor
1,1650,female,group A,some high school,free/reduced,completed,34,unemployed,middle class
2,717,female,group C,some high school,standard,none,59,employed,middle class
3,258,female,group D,master's degree,standard,none,68,employed,poor
4,1312,male,group D,some college,standard,completed,74,employed,poor


In [20]:
train['gender'] = train['gender'].map({'male':0, 'female':1})
train['race/ethnicity'] = train['race/ethnicity'].map({'group A':0, 'group B':1, 'group C':2, 'group D':3, 'group E':4})
train['parental level of education'] = train['parental level of education'].map({'some college':0, "master's degree":1, 'high school':2, 'some high school':3, "associate's degree":4, "bachelor's degree":5})
train['lunch'] = train['lunch'].map({'standard':0, 'free/reduced':1})
train['test preparation course'] = train['test preparation course'].map({'none':0, 'completed':1})
train['parental_job_status'] = train['parental_job_status'].map({'employed':0, 'unemployed':1})
train['parental_wealth_classification'] = train['parental_wealth_classification'].map({'middle class':0, 'poor':1, 'wealthy':2})

In [21]:
test['gender'] = test['gender'].map({'male':0, 'female':1})
test['race/ethnicity'] = test['race/ethnicity'].map({'group A':0, 'group B':1, 'group C':2, 'group D':3, 'group E':4})
test['parental level of education'] = test['parental level of education'].map({'some college':0, "master's degree":1, 'high school':2, 'some high school':3, "associate's degree":4, "bachelor's degree":5})
test['lunch'] = test['lunch'].map({'standard':0, 'free/reduced':1})
test['test preparation course'] = test['test preparation course'].map({'none':0, 'completed':1})
test['parental_job_status'] = test['parental_job_status'].map({'employed':0, 'unemployed':1})
test['parental_wealth_classification'] = test['parental_wealth_classification'].map({'middle class':0, 'poor':1, 'wealthy':2})

In [22]:
train.head()

,student_id,gender,race/ethnicity,parental level of education,lunch,test preparation course,score,parental_job_status,parental_wealth_classification
0,1014,1,3,2,0,1,88,0,1
1,1650,1,0,3,1,1,34,1,0
2,717,1,2,3,0,0,59,0,0
3,258,1,3,1,0,0,68,0,1
4,1312,0,3,0,0,1,74,0,1


In [23]:
test.head()

,student_id,gender,race/ethnicity,parental level of education,lunch,test preparation course,parental_job_status,parental_wealth_classification
0,810,1,0,0,0,0,0,0
1,1609,1,0,4,1,1,0,1
2,15,1,0,3,1,0,1,0
3,134,0,3,1,0,0,0,2
4,846,0,2,4,0,1,0,1


In [24]:
X = train.drop('score', axis=1)
y = train['score']

# start stacking

In [96]:
def get_models():
    models = dict()
    models['knn'] = KNeighborsRegressor()
    models['rfc'] = RandomForestRegressor()
    models['xgb'] = XGBRegressor()
    
    return models

In [97]:
def evaluate_model(model):
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
    
    return scores

In [98]:
models = get_models()
results, names = list(), list()

for name, model in models.items():
    scores = evaluate_model(model)
    results.append(scores)
    names.append(name)
    
    print('>%s %.3f(%.3f)' %(name, np.mean(scores), np.std(scores)))

>knn -12.489(0.730)
>rfc -5.567(0.343)
>xgb -5.962(0.396)


In [99]:
def get_stacking():
    level0 = list()
    level0.append(('knn', KNeighborsRegressor()))
    level0.append(('rfc', RandomForestRegressor()))
    level0.append(('xgb', XGBRegressor()))
    
    level1 = CatBoostRegressor()
    
    model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
    
    return model

In [100]:
def get_models_final():
    models = dict()
    models['knn'] = KNeighborsRegressor()
    models['rfc'] = RandomForestRegressor()
    models['xgb'] = XGBRegressor()
    models['stacking'] = get_stacking()
    
    return models

In [101]:
models = get_models_final()

results, names = list(), list()

for name, model in models.items():
    scores = evaluate_model(model)
    results.append(scores)
    names.append(name)
    
    print('>%s %.3f(%.3f)' %(name, np.mean(scores), np.std(scores)))

>knn -12.489(0.730)
>rfc -5.600(0.323)
>xgb -5.962(0.396)
>stacking -5.657(0.421)


### getting prediction from the stacked models

In [120]:
level0 = list()
level0.append(('knn', KNeighborsRegressor()))
level0.append(('rfc', RandomForestRegressor()))
level0.append(('xgb', XGBRegressor()))

level1 = CatBoostRegressor()

final_model = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)





In [121]:
final_model.fit(X, y)

Learning rate set to 0.040318
0:	learn: 13.7094699	total: 2.64ms	remaining: 2.64s
1:	learn: 13.3239468	total: 4.46ms	remaining: 2.22s
2:	learn: 12.9637305	total: 6.24ms	remaining: 2.07s
3:	learn: 12.6094129	total: 7.97ms	remaining: 1.98s
4:	learn: 12.2866442	total: 9.73ms	remaining: 1.94s
5:	learn: 11.9745942	total: 11.5ms	remaining: 1.91s
6:	learn: 11.6707117	total: 13.3ms	remaining: 1.88s
7:	learn: 11.3866119	total: 16.2ms	remaining: 2.01s
8:	learn: 11.1155870	total: 18.6ms	remaining: 2.05s
9:	learn: 10.8647555	total: 20.5ms	remaining: 2.03s
10:	learn: 10.6180498	total: 23.1ms	remaining: 2.08s
11:	learn: 10.3851747	total: 25.5ms	remaining: 2.1s
12:	learn: 10.1725207	total: 26.3ms	remaining: 2s
13:	learn: 9.9586857	total: 28.1ms	remaining: 1.98s
14:	learn: 9.7688855	total: 29.8ms	remaining: 1.96s
15:	learn: 9.5836618	total: 31.6ms	remaining: 1.94s
16:	learn: 9.4179786	total: 33.3ms	remaining: 1.93s
17:	learn: 9.2535242	total: 36.8ms	remaining: 2.01s
18:	learn: 9.0996410	total: 38.8ms	

238:	learn: 6.2978674	total: 450ms	remaining: 1.43s
239:	learn: 6.2943707	total: 453ms	remaining: 1.43s
240:	learn: 6.2909370	total: 454ms	remaining: 1.43s
241:	learn: 6.2869624	total: 456ms	remaining: 1.43s
242:	learn: 6.2858455	total: 458ms	remaining: 1.43s
243:	learn: 6.2846791	total: 460ms	remaining: 1.42s
244:	learn: 6.2809810	total: 461ms	remaining: 1.42s
245:	learn: 6.2800613	total: 463ms	remaining: 1.42s
246:	learn: 6.2780018	total: 465ms	remaining: 1.42s
247:	learn: 6.2756216	total: 467ms	remaining: 1.41s
248:	learn: 6.2740557	total: 468ms	remaining: 1.41s
249:	learn: 6.2715113	total: 470ms	remaining: 1.41s
250:	learn: 6.2674107	total: 473ms	remaining: 1.41s
251:	learn: 6.2662375	total: 474ms	remaining: 1.41s
252:	learn: 6.2642332	total: 476ms	remaining: 1.41s
253:	learn: 6.2637323	total: 478ms	remaining: 1.4s
254:	learn: 6.2603570	total: 480ms	remaining: 1.4s
255:	learn: 6.2593461	total: 482ms	remaining: 1.4s
256:	learn: 6.2564152	total: 484ms	remaining: 1.4s
257:	learn: 6.25

426:	learn: 5.8652465	total: 774ms	remaining: 1.04s
427:	learn: 5.8638945	total: 776ms	remaining: 1.04s
428:	learn: 5.8617385	total: 777ms	remaining: 1.03s
429:	learn: 5.8609658	total: 779ms	remaining: 1.03s
430:	learn: 5.8571131	total: 781ms	remaining: 1.03s
431:	learn: 5.8549485	total: 783ms	remaining: 1.03s
432:	learn: 5.8536336	total: 786ms	remaining: 1.03s
433:	learn: 5.8473594	total: 788ms	remaining: 1.03s
434:	learn: 5.8463183	total: 790ms	remaining: 1.03s
435:	learn: 5.8433730	total: 792ms	remaining: 1.02s
436:	learn: 5.8413819	total: 795ms	remaining: 1.02s
437:	learn: 5.8394423	total: 797ms	remaining: 1.02s
438:	learn: 5.8372234	total: 799ms	remaining: 1.02s
439:	learn: 5.8354406	total: 801ms	remaining: 1.02s
440:	learn: 5.8325710	total: 803ms	remaining: 1.02s
441:	learn: 5.8304810	total: 804ms	remaining: 1.01s
442:	learn: 5.8268911	total: 806ms	remaining: 1.01s
443:	learn: 5.8249415	total: 809ms	remaining: 1.01s
444:	learn: 5.8245270	total: 812ms	remaining: 1.01s
445:	learn: 

665:	learn: 5.3692634	total: 1.19s	remaining: 596ms
666:	learn: 5.3654382	total: 1.19s	remaining: 595ms
667:	learn: 5.3652387	total: 1.19s	remaining: 593ms
668:	learn: 5.3646908	total: 1.19s	remaining: 591ms
669:	learn: 5.3632110	total: 1.2s	remaining: 589ms
670:	learn: 5.3613450	total: 1.2s	remaining: 588ms
671:	learn: 5.3565753	total: 1.2s	remaining: 586ms
672:	learn: 5.3534344	total: 1.2s	remaining: 584ms
673:	learn: 5.3506049	total: 1.2s	remaining: 583ms
674:	learn: 5.3485364	total: 1.21s	remaining: 581ms
675:	learn: 5.3483161	total: 1.21s	remaining: 579ms
676:	learn: 5.3459275	total: 1.21s	remaining: 577ms
677:	learn: 5.3442867	total: 1.21s	remaining: 576ms
678:	learn: 5.3411083	total: 1.21s	remaining: 574ms
679:	learn: 5.3408436	total: 1.22s	remaining: 572ms
680:	learn: 5.3372007	total: 1.22s	remaining: 570ms
681:	learn: 5.3366502	total: 1.22s	remaining: 569ms
682:	learn: 5.3358628	total: 1.22s	remaining: 567ms
683:	learn: 5.3321530	total: 1.22s	remaining: 565ms
684:	learn: 5.330

851:	learn: 5.0560866	total: 1.51s	remaining: 262ms
852:	learn: 5.0548966	total: 1.51s	remaining: 260ms
853:	learn: 5.0516386	total: 1.51s	remaining: 258ms
854:	learn: 5.0491117	total: 1.51s	remaining: 256ms
855:	learn: 5.0479607	total: 1.51s	remaining: 255ms
856:	learn: 5.0472343	total: 1.51s	remaining: 253ms
857:	learn: 5.0462830	total: 1.52s	remaining: 251ms
858:	learn: 5.0450729	total: 1.52s	remaining: 249ms
859:	learn: 5.0446124	total: 1.52s	remaining: 248ms
860:	learn: 5.0404656	total: 1.52s	remaining: 246ms
861:	learn: 5.0386016	total: 1.53s	remaining: 244ms
862:	learn: 5.0384419	total: 1.53s	remaining: 243ms
863:	learn: 5.0371709	total: 1.53s	remaining: 241ms
864:	learn: 5.0359427	total: 1.53s	remaining: 239ms
865:	learn: 5.0334062	total: 1.53s	remaining: 237ms
866:	learn: 5.0303260	total: 1.53s	remaining: 236ms
867:	learn: 5.0301853	total: 1.54s	remaining: 234ms
868:	learn: 5.0278941	total: 1.54s	remaining: 232ms
869:	learn: 5.0265054	total: 1.54s	remaining: 230ms
870:	learn: 

StackingRegressor(cv=5,
                  estimators=[('knn', KNeighborsRegressor()),
                              ('rfc', RandomForestRegressor()),
                              ('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, gamma=None,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=None,
                                            max_delta_step=None, max_depth=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=Non

In [122]:
final_pred = final_model.predict(test)

In [81]:
test.shape

(800, 8)

In [82]:
final_pred.shape

(800,)

In [123]:
submit['score'] = final_pred

In [124]:
submit.to_csv('tro2.csv', index=None)

In [85]:
from sklearn.model_selection import train_test_split

In [112]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [113]:
final_model.fit(X_train, y_train)

Learning rate set to 0.037838
0:	learn: 13.7229734	total: 1.86ms	remaining: 1.86s
1:	learn: 13.3778466	total: 3.46ms	remaining: 1.73s
2:	learn: 13.0282301	total: 5.15ms	remaining: 1.71s
3:	learn: 12.6955783	total: 7.06ms	remaining: 1.76s
4:	learn: 12.4039253	total: 8.68ms	remaining: 1.73s
5:	learn: 12.0973314	total: 10.2ms	remaining: 1.69s
6:	learn: 11.8146404	total: 11.8ms	remaining: 1.67s
7:	learn: 11.5425476	total: 13.3ms	remaining: 1.65s
8:	learn: 11.2768214	total: 14.9ms	remaining: 1.64s
9:	learn: 11.0186587	total: 16.5ms	remaining: 1.63s
10:	learn: 10.7841581	total: 18ms	remaining: 1.62s
11:	learn: 10.5567440	total: 19.5ms	remaining: 1.61s
12:	learn: 10.3536667	total: 21.2ms	remaining: 1.61s
13:	learn: 10.1462154	total: 22.9ms	remaining: 1.61s
14:	learn: 9.9546955	total: 24.5ms	remaining: 1.61s
15:	learn: 9.7675883	total: 26.5ms	remaining: 1.63s
16:	learn: 9.5907559	total: 28.5ms	remaining: 1.65s
17:	learn: 9.4241517	total: 30.1ms	remaining: 1.64s
18:	learn: 9.2652877	total: 31.8

246:	learn: 6.0914946	total: 452ms	remaining: 1.38s
247:	learn: 6.0885260	total: 455ms	remaining: 1.38s
248:	learn: 6.0846567	total: 457ms	remaining: 1.38s
249:	learn: 6.0804764	total: 459ms	remaining: 1.38s
250:	learn: 6.0788544	total: 461ms	remaining: 1.38s
251:	learn: 6.0757684	total: 464ms	remaining: 1.38s
252:	learn: 6.0735436	total: 466ms	remaining: 1.38s
253:	learn: 6.0726065	total: 467ms	remaining: 1.37s
254:	learn: 6.0699776	total: 469ms	remaining: 1.37s
255:	learn: 6.0679649	total: 471ms	remaining: 1.37s
256:	learn: 6.0637598	total: 474ms	remaining: 1.37s
257:	learn: 6.0629951	total: 476ms	remaining: 1.37s
258:	learn: 6.0617612	total: 478ms	remaining: 1.37s
259:	learn: 6.0605638	total: 480ms	remaining: 1.36s
260:	learn: 6.0565296	total: 481ms	remaining: 1.36s
261:	learn: 6.0555004	total: 483ms	remaining: 1.36s
262:	learn: 6.0532447	total: 484ms	remaining: 1.36s
263:	learn: 6.0508130	total: 486ms	remaining: 1.36s
264:	learn: 6.0446395	total: 488ms	remaining: 1.35s
265:	learn: 

414:	learn: 5.5567718	total: 738ms	remaining: 1.04s
415:	learn: 5.5519601	total: 740ms	remaining: 1.04s
416:	learn: 5.5476546	total: 742ms	remaining: 1.04s
417:	learn: 5.5462533	total: 744ms	remaining: 1.04s
418:	learn: 5.5408366	total: 746ms	remaining: 1.03s
419:	learn: 5.5365265	total: 747ms	remaining: 1.03s
420:	learn: 5.5304603	total: 749ms	remaining: 1.03s
421:	learn: 5.5258837	total: 750ms	remaining: 1.03s
422:	learn: 5.5209354	total: 752ms	remaining: 1.02s
423:	learn: 5.5199338	total: 754ms	remaining: 1.02s
424:	learn: 5.5175075	total: 756ms	remaining: 1.02s
425:	learn: 5.5153462	total: 758ms	remaining: 1.02s
426:	learn: 5.5149927	total: 761ms	remaining: 1.02s
427:	learn: 5.5123761	total: 763ms	remaining: 1.02s
428:	learn: 5.5079652	total: 765ms	remaining: 1.02s
429:	learn: 5.5036842	total: 767ms	remaining: 1.02s
430:	learn: 5.5009382	total: 769ms	remaining: 1.01s
431:	learn: 5.4997144	total: 770ms	remaining: 1.01s
432:	learn: 5.4959847	total: 772ms	remaining: 1.01s
433:	learn: 

598:	learn: 5.0838726	total: 1.04s	remaining: 696ms
599:	learn: 5.0826356	total: 1.04s	remaining: 694ms
600:	learn: 5.0805422	total: 1.04s	remaining: 692ms
601:	learn: 5.0790072	total: 1.04s	remaining: 690ms
602:	learn: 5.0787404	total: 1.04s	remaining: 688ms
603:	learn: 5.0762895	total: 1.05s	remaining: 686ms
604:	learn: 5.0733600	total: 1.05s	remaining: 685ms
605:	learn: 5.0695819	total: 1.05s	remaining: 683ms
606:	learn: 5.0676508	total: 1.05s	remaining: 681ms
607:	learn: 5.0650221	total: 1.05s	remaining: 679ms
608:	learn: 5.0623155	total: 1.05s	remaining: 677ms
609:	learn: 5.0614524	total: 1.05s	remaining: 675ms
610:	learn: 5.0609848	total: 1.06s	remaining: 673ms
611:	learn: 5.0593770	total: 1.06s	remaining: 671ms
612:	learn: 5.0558243	total: 1.06s	remaining: 670ms
613:	learn: 5.0550281	total: 1.06s	remaining: 668ms
614:	learn: 5.0519562	total: 1.06s	remaining: 666ms
615:	learn: 5.0500278	total: 1.06s	remaining: 664ms
616:	learn: 5.0481781	total: 1.07s	remaining: 662ms
617:	learn: 

793:	learn: 4.6808835	total: 1.34s	remaining: 349ms
794:	learn: 4.6785046	total: 1.34s	remaining: 347ms
795:	learn: 4.6784005	total: 1.35s	remaining: 345ms
796:	learn: 4.6760292	total: 1.35s	remaining: 343ms
797:	learn: 4.6741339	total: 1.35s	remaining: 342ms
798:	learn: 4.6720597	total: 1.35s	remaining: 340ms
799:	learn: 4.6702868	total: 1.35s	remaining: 338ms
800:	learn: 4.6701120	total: 1.35s	remaining: 336ms
801:	learn: 4.6678189	total: 1.35s	remaining: 335ms
802:	learn: 4.6652306	total: 1.36s	remaining: 333ms
803:	learn: 4.6632444	total: 1.36s	remaining: 331ms
804:	learn: 4.6630722	total: 1.36s	remaining: 330ms
805:	learn: 4.6604544	total: 1.36s	remaining: 328ms
806:	learn: 4.6603652	total: 1.36s	remaining: 326ms
807:	learn: 4.6584513	total: 1.36s	remaining: 324ms
808:	learn: 4.6560874	total: 1.37s	remaining: 323ms
809:	learn: 4.6555223	total: 1.37s	remaining: 321ms
810:	learn: 4.6547991	total: 1.37s	remaining: 319ms
811:	learn: 4.6536998	total: 1.37s	remaining: 318ms
812:	learn: 

988:	learn: 4.3393111	total: 1.65s	remaining: 18.3ms
989:	learn: 4.3386243	total: 1.65s	remaining: 16.7ms
990:	learn: 4.3360606	total: 1.65s	remaining: 15ms
991:	learn: 4.3332565	total: 1.65s	remaining: 13.3ms
992:	learn: 4.3315518	total: 1.66s	remaining: 11.7ms
993:	learn: 4.3299051	total: 1.66s	remaining: 10ms
994:	learn: 4.3280676	total: 1.66s	remaining: 8.34ms
995:	learn: 4.3278318	total: 1.66s	remaining: 6.67ms
996:	learn: 4.3266163	total: 1.66s	remaining: 5ms
997:	learn: 4.3241959	total: 1.66s	remaining: 3.33ms
998:	learn: 4.3217563	total: 1.66s	remaining: 1.67ms
999:	learn: 4.3200850	total: 1.67s	remaining: 0us


StackingRegressor(cv=5,
                  estimators=[('knn', KNeighborsRegressor()),
                              ('rfc', RandomForestRegressor()),
                              ('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, gamma=None,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=None,
                                            max_delta_step=None, max_depth=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=Non

In [114]:
pred = final_model.predict(X_test)

In [89]:
from sklearn.metrics import mean_absolute_error

In [115]:
score = mean_absolute_error(y_test, pred)

In [91]:
import math

In [116]:
math.sqrt(score)

2.3923071505319995

In [117]:
my_pred = final_model.predict(test)

In [118]:
submit['score'] = my_pred

In [119]:
submit.to_csv('tro.csv', index=None)